# Install and import

In [1]:
!pip install -qU sentence-transformers
!pip install udocker==1.3.17 --quiet
!pip install redisvl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.6/119.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 17.2 MB/s eta 0:00:00


In [2]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import top_k_accuracy_score
import numpy as np
from pprint import pprint
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer
import  torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import time

In [3]:
import warnings
warnings.filterwarnings('ignore')

# Set up Huggingface token

# Data Preparation and Feature Engineering

In [31]:
!wget -nv https://files.grouplens.org/datasets/movielens/ml-100k.zip
!wget -nv https://files.grouplens.org/datasets/movielens/ml-100k-README.txt

!unzip -qo ml-100k.zip

2025-02-25 00:46:06 URL:https://files.grouplens.org/datasets/movielens/ml-100k.zip [4924029/4924029] -> "ml-100k.zip" [1]
2025-02-25 00:46:06 URL:https://files.grouplens.org/datasets/movielens/ml-100k-README.txt [6748/6748] -> "ml-100k-README.txt" [1]


## Spark Installation

In [30]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840626 sha256=3f4c2ae13fb81422eef995793f5ad06093a97ab5fd5f460f200d7f1e0918313e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.4
    Uninstalling pyspark-3.5.4:
      Successfully uninstalled pyspark-3.5.4


In [32]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [33]:
import pandas as pd
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, LongType
import time
from sklearn.model_selection import train_test_split
from pyspark.ml.feature import StandardScaler

## Read the rating file

In [34]:
rating_schema = StructType([ \
StructField('user_id', IntegerType(), True), \
StructField('movie_id', IntegerType(), True), \
StructField('rating', IntegerType(), True), \
StructField('timestamp', LongType(), True) ])

In [35]:
rating=spark.read.format("csv").option("header","false").option("sep","\t").schema(rating_schema).load('./ml-100k/u.data')
rating.createOrReplaceTempView("rating")

In [36]:
rating.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|    196|     242|     3|881250949|
|    186|     302|     3|891717742|
|     22|     377|     1|878887116|
|    244|      51|     2|880606923|
|    166|     346|     1|886397596|
|    298|     474|     4|884182806|
|    115|     265|     2|881171488|
|    253|     465|     5|891628467|
|    305|     451|     3|886324817|
|      6|      86|     3|883603013|
|     62|     257|     2|879372434|
|    286|    1014|     5|879781125|
|    200|     222|     5|876042340|
|    210|      40|     3|891035994|
|    224|      29|     3|888104457|
|    303|     785|     3|879485318|
|    122|     387|     5|879270459|
|    194|     274|     2|879539794|
|    291|    1042|     4|874834944|
|    234|    1184|     2|892079237|
+-------+--------+------+---------+
only showing top 20 rows



In [ ]:
spark.sql("select count(*) from rating").show()

+--------+
|count(1)|
+--------+
|  100000|
+--------+



### Create negetive sample

In [ ]:
negetive_pair=spark.sql(f"""

    SELECT user_id, movie_id
    FROM (
    SELECT u1.user_id, m.movie_id,
           ROW_NUMBER() OVER (PARTITION BY u1.user_id ORDER BY RAND()) AS rn
    FROM (SELECT DISTINCT user_id FROM rating) u1  -- Get all users
    CROSS JOIN (SELECT DISTINCT movie_id FROM rating) m  -- Get all movies
    WHERE m.movie_id NOT IN (
        SELECT movie_id FROM rating u2 WHERE u2.user_id = u1.user_id
    )
 ) subquery
WHERE rn <= 50;

""")
negetive_pair.createOrReplaceTempView("negetive_pair")

In [ ]:
negetive_pair.show()

+-------+--------+
|user_id|movie_id|
+-------+--------+
|      1|    1439|
|      1|     429|
|      1|    1620|
|      1|    1355|
|      1|     913|
|      1|     344|
|      1|    1067|
|      1|     555|
|      1|    1338|
|      1|     382|
|      1|     416|
|      1|     365|
|      1|    1482|
|      1|    1235|
|      1|     419|
|      1|     477|
|      1|     875|
|      1|     284|
|      1|    1024|
|      1|    1380|
+-------+--------+
only showing top 20 rows



In [ ]:
rating_pos_neg=spark.sql(f"""
with mx_min_time as (
select max(timestamp) as max_time ,
       min(timestamp)
       as min_time
       from rating
)
   select user_id,
       movie_id,
       0 as  rating,
       FLOOR(RAND() * (max_time - min_time + 1)) + min_time as timestamp,
       0 as flag
       from negetive_pair
   join  mx_min_time mn on 1 = 1
   union
   select user_id,
       movie_id,
       rating,
       timestamp,
       1 as flag
       from rating
""")
rating_pos_neg.createOrReplaceTempView("rating_pos_neg")

In [ ]:
spark.sql("select count(*) from rating_pos_neg where flag=0").show()

+--------+
|count(1)|
+--------+
|   47150|
+--------+



## Read Item ( movie ) file

In [37]:
item_schema = StructType([ \
StructField('movie_id', IntegerType(), True), \
StructField('movie_title', StringType(), True), \
StructField('release_date', StringType(), True), \
StructField('video_release_date', StringType(), True), \
StructField('imdb_url', StringType(), True), \

StructField('unknown', IntegerType(), True), \
StructField('Action', IntegerType(), True), \
StructField('Adventure', IntegerType(), True), \
StructField('Animation', IntegerType(), True), \
StructField('Children', IntegerType(), True), \

StructField('Comedy', IntegerType(), True), \
StructField('Crime', IntegerType(), True), \
StructField('Documentary', IntegerType(), True), \
StructField('Drama', IntegerType(), True), \
StructField('Fantasy', IntegerType(), True), \

StructField('Film_Noir', IntegerType(), True), \
StructField('Horror', IntegerType(), True), \
StructField('Musical', IntegerType(), True), \
StructField('Mystery', IntegerType(), True), \
StructField('Romance', IntegerType(), True), \

StructField('Sci_Fi', IntegerType(), True), \
StructField('Thriller', IntegerType(), True), \
StructField('War', IntegerType(), True), \
StructField('Western', IntegerType(), True) ])

In [42]:
items=spark.read.format("csv").option("header","false").option("sep","|").schema(item_schema).load('./ml-100k/u.item')
items.createOrReplaceTempView("items")

In [43]:
items.show(), items.count()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|         movie_title|release_date|video_release_date|            imdb_url|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|       1|    Toy Story (1995)| 01-Jan-1995|              NULL|http://us.imdb.co...|      0|     0|        0|        1|       1|     1|    0|          0|    0|      0|        0|     0|      0|      0|      0|     0|       0|  0|      0|
|       2|    GoldenEye (1995)| 01-Jan-1995|        

(None, 1682)

In [44]:
spark.sql("select max(movie_id) from items").show()

+-------------+
|max(movie_id)|
+-------------+
|         1682|
+-------------+



In [ ]:
spark.sql("select min(movie_id) from items").show()

+-------------+
|min(movie_id)|
+-------------+
|            1|
+-------------+



## Read Genre file

In [ ]:
genre_schema = StructType([ \
StructField('genre', StringType(), True), \
StructField('genre_id', IntegerType(), True) ])

In [ ]:
genre=spark.read.format("csv").option("header","false").option("sep","|").schema(genre_schema).load('./ml-100k/u.genre')
genre.createOrReplaceTempView("genre")

In [ ]:
genre.show()

+-----------+--------+
|      genre|genre_id|
+-----------+--------+
|    unknown|       0|
|     Action|       1|
|  Adventure|       2|
|  Animation|       3|
| Children's|       4|
|     Comedy|       5|
|      Crime|       6|
|Documentary|       7|
|      Drama|       8|
|    Fantasy|       9|
|  Film-Noir|      10|
|     Horror|      11|
|    Musical|      12|
|    Mystery|      13|
|    Romance|      14|
|     Sci-Fi|      15|
|   Thriller|      16|
|        War|      17|
|    Western|      18|
+-----------+--------+



## Read User file

In [ ]:
user_schema = StructType([ \
StructField('user_id', IntegerType(), True), \
StructField('age', IntegerType(), True), \
StructField('gender', StringType(), True), \
StructField('occupation', StringType(), True), \
StructField('zip_code', IntegerType(), True) ])

In [ ]:
user=spark.read.format("csv").option("header","false").option("sep","|").schema(user_schema).load('./ml-100k/u.user')
user.createOrReplaceTempView("user")

In [ ]:
user.show(),user.count()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      1| 24|     M|   technician|   85711|
|      2| 53|     F|        other|   94043|
|      3| 23|     M|       writer|   32067|
|      4| 24|     M|   technician|   43537|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|    5201|
|      9| 29|     M|      student|    1002|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     12| 28|     F|        other|    6405|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     16| 21|     M|entertainment|   10309|
|     17| 30|     M|   programmer|    6355|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|    2138|
|     20| 42|     F|    homemake

(None, 943)

## Feature Engineering
- Encode the gender column to one-hot
- normalize the age
- label occupation with number

In [ ]:
user_feature=spark.sql(f"""
        with OccupationLabelEncoder as (
                    select occupation,
                           DENSE_RANK() OVER (ORDER BY occupation ASC)  as occupation_label
                     from (
                      select distinct occupation from user
                     )
        )
        select u.*,
                -- Encode the gender column to one-hot
                CASE WHEN u.gender = 'M' THEN 1 ELSE 0 END AS gen_M,
                CASE WHEN u.gender = 'F' THEN 1 ELSE 0 END AS gen_F,
                occupation_label,
                (u.age  - AVG(u.age ) OVER ()) / STDDEV(u.age ) OVER () AS standardized_user_age

        from user u
        INNER JOIN OccupationLabelEncoder o ON o.occupation = u.occupation
        """)

user_feature.createOrReplaceTempView("user_feature")

In [ ]:
spark.sql("select count(*) from user_feature").show(), spark.sql("select * from user_feature").show()

+--------+
|count(1)|
+--------+
|     943|
+--------+

+-------+---+------+-------------+--------+-----+-----+----------------+---------------------+
|user_id|age|gender|   occupation|zip_code|gen_M|gen_F|occupation_label|standardized_user_age|
+-------+---+------+-------------+--------+-----+-----+----------------+---------------------+
|      1| 24|     M|   technician|   85711|    1|    0|              20|  -0.8244219136993041|
|      2| 53|     F|        other|   94043|    0|    1|              14|    1.554042700071649|
|      3| 23|     M|       writer|   32067|    1|    0|              21|  -0.9064379348638197|
|      4| 24|     M|   technician|   43537|    1|    0|              20|  -0.8244219136993041|
|      5| 33|     F|        other|   15213|    0|    1|              14| -0.08627772321866344|
|      6| 42|     M|    executive|   98101|    1|    0|               7|   0.6518664672619772|
|      7| 57|     M|administrator|   91344|    1|    0|               1|   1.882106784729

(None, None)

- normalize timestamp

In [ ]:
rating_feature = spark.sql(f"""
                    SELECT
                    r.user_id,
                    r.movie_id,
                    r.rating,
                    (r.timestamp  - AVG(r.timestamp ) OVER ()) / STDDEV(r.timestamp ) OVER () AS standardized_timestamp,
                    flag
                    from rating_pos_neg r
                """)
rating_feature.createOrReplaceTempView("rating_feature")

In [ ]:
spark.sql("select * from rating_feature where flag =1").show()

+-------+--------+------+----------------------+----+
|user_id|movie_id|rating|standardized_timestamp|flag|
+-------+--------+------+----------------------+----+
|     92|    1049|     1|    1.2284163523601754|   1|
|    225|     480|     5|   -0.7729238301424504|   1|
|    184|     218|     3|    1.1645170582225226|   1|
|    306|     287|     4|   -1.3402505926003299|   1|
|     90|     923|     5|    1.4399440064253095|   1|
|    313|     566|     4|     1.371241605566614|   1|
|    121|     180|     3|    1.4407612782216932|   1|
|      5|     424|     1|    -1.502553035867774|   1|
|    291|     428|     5|   -1.6453123318396161|   1|
|    296|     248|     5|   0.09704219125290574|   1|
|    293|    1161|     2|    0.9767766098853315|   1|
|     69|     175|     3|  -0.28621586682542716|   1|
|    267|     739|     4|   -0.8789546625241332|   1|
|     92|     763|     3|    0.5167818871228104|   1|
|     17|     508|     3|   0.29809292164020945|   1|
|    279|    1493|     1|   

In [ ]:
pos_neg_records=spark.sql(f"""
          select
          (u.user_id -1 ) as user_id,
          (m.movie_id -1) as item_id,
          rating,
          gen_M,
          gen_F,
          standardized_user_age as age,
          standardized_timestamp as timestamp,
          occupation_label as occupation,
          zip_code,
          LOWER(REPLACE(REPLACE(REPLACE(movie_title, ':', ''), '&', ''), ',', '')) as movie_title,
          release_date,
          video_release_date,
          imdb_url,
          unknown,
          Action,
          Adventure,
          Animation,
          Children,
          Comedy,
          Crime,
          Documentary,
          Drama,
          Fantasy,
          Film_Noir,
          Horror,
          Musical,
          Mystery,
          Romance,
          Sci_Fi,
          Thriller,
          War,
          Western,
          flag
          from rating_feature r
          INNER JOIN Items m ON r.movie_id = m.movie_id
          INNER JOIN user_feature u ON r.user_id = u.user_id
          order by user_id, item_id
""");
pos_neg_records.createOrReplaceTempView("pos_neg_records")

In [ ]:
spark.sql("select * from pos_neg_records where user_id = 0").show()

+-------+-------+------+-----+-----+-------------------+-------------------+----------+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+----+
|user_id|item_id|rating|gen_M|gen_F|                age|          timestamp|occupation|zip_code|         movie_title|release_date|video_release_date|            imdb_url|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|flag|
+-------+-------+------+-----+-----+-------------------+-------------------+----------+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+----+
|      0|      0

In [ ]:
spark.sql("select count(*) from pos_neg_records where flag=0").show()

+--------+
|count(1)|
+--------+
|   47150|
+--------+



In [ ]:
spark.sql("select count(*) from pos_neg_records where flag=1").show()

+--------+
|count(1)|
+--------+
|  100000|
+--------+



In [ ]:
pos_neg_records.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("./pos_neg_records")

In [ ]:
 %%sh
 filename=`ls ./pos_neg_records |grep -v _SUCCESS`
 echo "mv ./pos_neg_records/$filename ./pos_neg_records.csv"
 mv ./pos_neg_records/$filename ./pos_neg_records.csv


mv ./pos_neg_records/part-00000-017fb65e-d98e-44f6-ba32-eeefcdaed41f-c000.csv ./pos_neg_records.csv


# Training

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
#!wget https://github.com/reza-rahim/published/raw/refs/heads/main/recom_two_tower/data/pos_neg_records.csv.zip
#!unzip pos_neg_records.csv.zip

--2025-02-25 00:17:47--  https://github.com/reza-rahim/published/raw/refs/heads/main/recom_two_tower/data/pos_neg_records.csv.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/reza-rahim/published/refs/heads/main/recom_two_tower/data/pos_neg_records.csv.zip [following]
--2025-02-25 00:17:48--  https://raw.githubusercontent.com/reza-rahim/published/refs/heads/main/recom_two_tower/data/pos_neg_records.csv.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5947109 (5.7M) [application/zip]
Saving to: ‘pos_neg_records.csv.zip’

pos_neg_records.csv 100%[===================>]   5.67M  --.-KB/s    in

In [6]:
df_pos_neg_records = pd.read_csv("./pos_neg_records.csv",
                  delimiter=",",
                  header=0)
df_pos_neg_records.head()

,user_id,item_id,rating,gen_M,gen_F,age,timestamp,occupation,zip_code,movie_title,...,Film_Noir,Horror,Musical,Mystery,Romance,Sci_Fi,Thriller,War,Western,flag
0,0,0,5,1,0,-0.824422,-1.627750,20,85711.0,toy story (1995),...,0,0,0,0,0,0,0,0,0,1
1,0,1,3,1,0,-0.824422,-1.267617,20,85711.0,goldeneye (1995),...,0,0,0,0,0,0,1,0,0,1
2,0,2,4,1,0,-0.824422,-0.959358,20,85711.0,four rooms (1995),...,0,0,0,0,0,0,1,0,0,1
3,0,3,3,1,0,-0.824422,-1.267627,20,85711.0,get shorty (1995),...,0,0,0,0,0,0,0,0,0,1
4,0,4,3,1,0,-0.824422,1.134971,20,85711.0,copycat (1995),...,0,0,0,0,0,0,1,0,0,1


In [7]:
df_pos_neg_records["user_id"].count()

147150

This code initializes a SentenceTransformer model (all-MiniLM-L6-v2) to generate embeddings for unique movie titles in the df_pos_neg_records DataFrame. It then stores these embeddings in a dictionary (Movie_titlEmbbeging), where each movie title is mapped to its corresponding embedding as a PyTorch tensor.

In [8]:
# Initialize the SentenceTransformer model using the 'all-MiniLM-L6-v2' pre-trained model
model_strans = SentenceTransformer('all-MiniLM-L6-v2')

# Create an empty dictionary to store movie title embeddings
Movie_titlEmbbeging = {}

# Loop through all unique movie titles in the "df_pos_neg_records" DataFrame
for i in df_pos_neg_records["movie_title"].unique():
    # For each movie title, encode it using the pre-trained model to obtain the embedding
    # Store the movie title and its corresponding embedding in the dictionary
    Movie_titlEmbbeging[i] = torch.tensor(model_strans.encode(i))


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Split the data into training and testing sets using an 80-20 split

In [9]:

# The 'train_test_split' function randomly divides the 'df_pos_neg_records' DataFrame into
# two subsets: 'df_train' for training and 'df_test' for testing.
# - 'train_size=.8' means 80% of the data will be used for training and 20% for testing.
# - 'random_state=42' ensures reproducibility of the split (same random split every time).
# - 'shuffle=True' shuffles the data before splitting to ensure randomness.
df_train, df_test = train_test_split(df_pos_neg_records, train_size=.8, random_state=42, shuffle=True)


In [10]:
df_train["user_id"].count(), df_test["user_id"].count()

(117720, 29430)

In [11]:
# Check if a GPU (CUDA) is available, otherwise default to using the CPU
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Print the selected device to confirm whether the model will run on the GPU or CPU
print(f"Device is {device}")


Device is cuda


This code defines a custom PyTorch dataset (cls_dataset) for a movie recommendation system. It loads user and item data from a DataFrame and prepares it for model training.

Key Features:
- Initialization (__init__):

   Takes a DataFrame (data) containing user and item information and stores it.
- Length (__len__):

   Returns the total number of samples (rows) in the dataset.
- Fetching Data (__getitem__):

   Retrieves a sample at a given index and returns user features, item features, a flag, and a rating as PyTorch tensors.

   User features: Includes user_id, age, occupation, and gender (gen_F, gen_M).

   Item features: Includes item_id, timestamp, genre indicators, and a precomputed movie title embedding (ts_title).

   Flag & Rating: Extracts and returns them as tensors.


This dataset structure is designed to be used with PyTorch DataLoader for training a recommendation model.

In [12]:
class cls_dataset(Dataset):
    """
    Custom PyTorch dataset for movie recommendation.
    Returns user features, item features, flag, and rating.
    """

    def __init__(self, data):
        """
        Initializes the dataset with given data.

        Args:
            data (pd.DataFrame): The dataset containing user and item information.
        """
        super().__init__()
        self.m_data = data  # Store the dataset

    def __len__(self):
        """
        Returns the total number of samples in the dataset.

        Returns:
            int: Number of rows in the dataset.
        """
        return len(self.m_data)



    def __getitem__(self, index) :
         """
         Retrieves a single data sample at the given index.

         Args:
            index (int): Index of the sample to fetch.

         Returns:
            tuple: (user_tensor, item_tensor, flag_tensor, rating_tensor)
         """
         # Retrieve the movie title embedding from a dictionary or precomputed embeddings
         ts_title = Movie_titlEmbbeging[self.m_data.iloc[index]["movie_title"]]

         # Extract user-related features and convert them to a tensor
         user_tensor = torch.tensor(
             self.m_data.loc[self.m_data.index[index], ["user_id", "age", "occupation", "gen_F", "gen_M"]],
             dtype=torch.float32
         )

         # Extract item-related features (item_id, timestamp, genres) and concatenate with movie title embedding

         item_features = torch.tensor(
            torch.cat(( torch.tensor(self.m_data.iloc[index][[
                "item_id", "timestamp","unknown",'Action', 'Adventure', 'Animation', 'Children','Comedy',
                'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film_Noir','Horror', 'Musical', 'Mystery',
                'Romance', 'Sci_Fi', 'Thriller', 'War','Western'
            ]], dtype=torch.float32), ts_title), dim=0)
         )

          # Extract flag and rating as tensors
         flag_tensor = torch.tensor([self.m_data.iloc[index]["flag"]])
         rating_tensor =  torch.tensor([self.m_data.iloc[index]["rating"]])

         return user_tensor,item_features,flag_tensor, rating_tensor


In [13]:
# Create an instance of the custom dataset class using the training data (df_train)
ds = cls_dataset(df_train)

# Create a DataLoader for the training dataset
# - ds: The dataset instance
# - batch_size=1500: Each batch will contain 1500 samples for training
ds_trainLoader = DataLoader(ds, batch_size=1500)

# Create a DataLoader for the test dataset
# - Selects only the 'user_id' and 'item_id' columns from df_test
# - Converts the DataFrame to a NumPy array and then to a PyTorch tensor
# - batch_size=1000: Each batch will contain 1000 samples for testing
ds_testLoader = DataLoader(torch.tensor(df_test[["user_id", "item_id"]].to_numpy()), batch_size=1000)


In [ ]:
#test the ds_trainLoader
'''
for i, (x1,x2,y,_) in enumerate(tqdm(ds_trainLoader)):
        x_user,x_item,y_flag = x1.to(device), x2.to(device), y.to(device)
        print(f"x_user: {x_user} \n x_item: {x_item} \n y_flag: {y_flag}" )
        break
'''

'\nfor i, (x1,x2,y,_) in enumerate(tqdm(ds_trainLoader)):\n        x_user,x_item,y_flag = x1.to(device), x2.to(device), y.to(device)\n        print(f"x_user: {x_user} \n x_item: {x_item} \n y_flag: {y_flag}" )\n        break\n'

This code defines a PyTorch-based recommendation model (cls_model) that predicts user-item interactions using embeddings and neural networks.

Key Components:
- Embeddings for Users & Items

  Uses nn.Embedding to map users and items to dense vector representations.

- User & Item Feature Processing

  UserModel: A small neural network that processes user features (input: 20 features → output: 16).
  
  ItemModel: A larger neural network that processes item features (input: 420 features → output: 16).

- Classification Model

  Merges user and item representations (32-dimensional input) and predicts interaction (binary classification).

- Forward Method (forward)

  Combines user and item features, processes them through separate models, and passes them to the classifier to produce logits.

- Prediction Method (predict)

  Computes cosine similarity between a given user’s embedding and all item embeddings to recommend items.

This model is designed for recommendation systems, where it learns user-item relationships and can predict interactions or similarities.

In [14]:
import torch
import torch.nn as nn

class cls_model(nn.Module):
    def __init__(self, user_count, item_count, user_embedding_size=16, item_embedding_size=16):
        """
        Initializes the recommendation model.

        Args:
            user_count (int): Total number of unique users.
            item_count (int): Total number of unique items.
            user_embedding_size (int): Size of the user embedding vector (default: 16).
            item_embedding_size (int): Size of the item embedding vector (default: 16).
        """
        super().__init__()

        # Embedding layers for users and items
        self.UserEmbedding = nn.Embedding(user_count, user_embedding_size)
        self.ItemEmbedding = nn.Embedding(item_count, item_embedding_size)

        # User model: A small neural network to process user features
        self.UserModel = nn.Sequential(
            nn.Linear(20, 16),  # Input: 20 user features → Output: 16 features
            nn.ReLU(),          # Activation function
            nn.Linear(16, 16)   # Further transformation to 16-dimensional vector
        )

        # Item model: A larger neural network to process item features
        self.ItemModel = nn.Sequential(
            nn.Linear(420, 128),  # Input: 420 item features → Output: 128 features
            nn.ReLU(),            # Activation function
            nn.Linear(128, 16)    # Further transformation to 16-dimensional vector
        )

        # Classification model: Combines user and item representations and predicts interaction
        self.ClassifyModel = nn.Sequential(
            nn.Linear(32, 32),  # Concatenated user-item vectors (16+16) → 32 features
            nn.ReLU(),          # Activation function
            nn.Linear(32, 2)    # Final output (binary classification, e.g., interaction or rating)
        )

    def forward(self, user_data, item_data):
        """
        Forward pass: Processes user and item data through the network.

        Args:
            user_data (torch.Tensor): User feature tensor of shape [batch_size, user_feature_dim].
            item_data (torch.Tensor): Item feature tensor of shape [batch_size, item_feature_dim].

        Returns:
            torch.Tensor: Classification logits (output before softmax/sigmoid).
        """

        # Extract user and item embeddings using the first column (user_id, item_id)
        user_embedding_slice = self.UserEmbedding.forward(user_data[:, 0].long())  # User embedding lookup
        item_embedding_slice = self.ItemEmbedding.forward(item_data[:, 0].long())  # Item embedding lookup

        # Concatenate user/item embeddings with their respective feature vectors (excluding the first column)
        user_input = torch.cat((user_data[:, 1:], user_embedding_slice), dim=1)  # Shape: [batch, 20]
        item_input = torch.cat((item_data[:, 1:], item_embedding_slice), dim=1)  # Shape: [batch, 420]

        # Process user and item inputs through their respective models
        user_logits = self.UserModel.forward(user_input)  # Output shape: [batch, 16]
        item_logits = self.ItemModel.forward(item_input)  # Output shape: [batch, 16]

        # Concatenate processed user and item representations and pass through the classifier
        logits = self.ClassifyModel.forward(torch.cat((user_logits, item_logits), dim=1))  # Shape: [batch, 2]

        return logits  # Output: raw prediction scores (logits)

    def predict(self, user_id: torch.tensor):
        """
        Computes similarity between a given user and all available items.

        Args:
            user_id (torch.Tensor): Tensor containing user IDs.

        Returns:
            torch.Tensor: Cosine similarity scores between the user and all items.
        """

        # Get embedding vector for the given user ID
        user_embedding_single = self.UserEmbedding.forward(user_id.long())

        # Get all item embeddings (weight matrix from the embedding layer)
        item_embedding_all = self.ItemEmbedding.weight.data

        # Compute dot product (unnormalized similarity) between user and all item embeddings
        res = user_embedding_single @ item_embedding_all.T  # Shape: [user_count, item_count]

        # Compute the L2 norm (Euclidean norm) for user and item embeddings
        user_norm = torch.linalg.norm(user_embedding_single, dim=1, ord=2)  # Shape: [user_count]
        item_norm = torch.linalg.norm(item_embedding_all, dim=1, ord=2)     # Shape: [item_count]

        # Reshape norms for proper broadcasting in cosine similarity calculation
        user_norm.unsqueeze_(dim=1)  # Shape: [user_count, 1]
        item_norm.unsqueeze_(dim=0)  # Shape: [1, item_count]

        # Compute cosine similarity (dot product divided by norms)
        res = res / (user_norm @ item_norm)  # Normalized similarity scores

        return res  # Shape: [user_count, item_count]


In [15]:
# Set the number of epochs for training. Here, the model will train for 6 epoch.
epochs = 6

# Initialize the model by specifying the number of users and items.
# - df_train["user_id"].max()+1: The total number of unique users in the training data.
# - df_train["item_id"].max()+1: The total number of unique items in the training data.
# These are used to define the embedding layers for users and items.
model = cls_model(df_train["user_id"].max() + 1, df_train["item_id"].max() + 1)

# Move the model to the appropriate device (GPU or CPU) for training.
model = model.to(device)


In [16]:
# Define the loss function for the model.
# - nn.CrossEntropyLoss(): This is commonly used for classification tasks, and it computes the cross-entropy loss
#   between the predicted logits (raw model outputs) and the true labels.
criterion = nn.CrossEntropyLoss()

# Define the optimizer for updating the model's parameters during training.
# - torch.optim.Adam(): Adam optimizer is an adaptive learning rate optimization algorithm.
# - model.parameters(): Passes the model parameters to the optimizer so it can update them during backpropagation.
# - lr=5e-3: The learning rate is set to 0.005, which controls the step size for parameter updates during training.
optim = torch.optim.Adam(model.parameters(), lr=5e-3)


This code defines a PyTorch-based recommendation training process

- Start Time & Training Mode: The script records the start time and sets the model to training mode, enabling necessary features like dropout and batch normalization.

- Epoch Loop: Iterates through the specified number of epochs, processing data in batches.

- Batch Processing:
  Retrieves user and item features along with labels from the DataLoader.
  Moves data to the appropriate device (CPU/GPU).
  Performs a forward pass to generate predictions.
  Computes the loss using cross-entropy.
  Resets gradients before backpropagation.
  Updates model parameters using the optimizer.

- Loss Logging: Every 10 iterations, the average loss for
  the last 10 batches is printed, and the accumulated loss is reset.

- Evaluation Phase:
  The model switches to evaluation mode.
  Initializes total accuracy tracking for validation.
  Iterates through the test dataset, moves batches to the device, and generates predictions.
  Computes Top-10 accuracy, handling potential errors if they arise.
  Logs the total validation accuracy.
  Switches the model back to training mode after evaluation.
- Completion & Timing: Once training is complete, records the end time and prints the total elapsed time for performance tracking.

In [17]:
# Record the start time to track the total time for training.
start_time = time.time()

# Set the model to training mode, enabling features like dropout and batch normalization.
model.train()

# Iterate over the specified number of epochs (1 in this case).
for j in range(epochs):
    # Initialize a variable to accumulate the loss for each epoch.
    loss_acc = 0

    # Iterate over the training DataLoader in batches.
    # x1, x2 are the input features (user and item data), y is the target output (labels).
    for i, (x1, x2, y, _) in enumerate(tqdm(ds_trainLoader)):
        # Move the data to the appropriate device (GPU/CPU).
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)

        # Forward pass: compute the model's predictions (logits).
        logits = model.forward(x1, x2)

        # Compute the loss using the cross-entropy loss function.
        loss = criterion.forward(logits, y.squeeze())

        # Zero the gradients of the model's parameters before backpropagation.
        optim.zero_grad()

        # Backward pass: compute the gradients of the loss with respect to model parameters.
        loss.backward()

        # Perform a step of optimization, updating the model parameters.
        optim.step()

        # Accumulate the loss over the batches for the epoch.
        loss_acc += loss.item()

        # Every 10 iterations, print the average loss over the last 10 batches.
        if i % 10 == 0:
            # Print the average loss for the last 10 batches.
            print(f"Train ce loss {loss_acc / 10}")
            # Reset the accumulated loss for the next batch of 10.
            loss_acc = 0

            # Switch the model to evaluation mode for validation.
            model.eval()

            # Initialize a variable to accumulate the total accuracy for the validation set.
            total_acc = 0.0
            for k, batch in enumerate(ds_testLoader):
                # Move the test batch to the appropriate device.
                batch = batch.to(device)

                # Get the predictions for the current batch.
                pred = model.predict(batch[:, 0])

                try:
                    # Calculate the top-k accuracy for the current batch using `top_k_accuracy_score`.
                    acc = top_k_accuracy_score(batch[:, 1].detach().cpu().numpy(),
                                                pred.detach().cpu().numpy(), k=10, labels=range(pred.shape[1]))
                except ValueError:
                    # Handle any potential ValueError, such as issues with batch sizes or data inconsistencies.
                    print(f" ValueError: {k} ")
                # Accumulate the validation accuracy for all batches.
                total_acc += acc

            # Print the total top-k accuracy for the entire validation set.
            print(f"Total val top-k-accuracy: {total_acc / (k + 1)}")
            # Switch the model back to training mode after evaluation.
            model.train()

# Record the end time to calculate the total elapsed time for training.
end_time = time.time()

# Print the total elapsed time for training.
print("Elapsed time:", end_time - start_time, "seconds")


  0%|          | 0/79 [00:00<?, ?it/s]

Train ce loss 0.06526185870170594
Total val top-k-accuracy: 0.006165116279069769
Train ce loss 0.6195835113525391
Total val top-k-accuracy: 0.006065116279069769
Train ce loss 0.5836091697216034
Total val top-k-accuracy: 0.005865116279069769
Train ce loss 0.5593260109424592
Total val top-k-accuracy: 0.005820930232558141
Train ce loss 0.534764289855957
Total val top-k-accuracy: 0.005876744186046513
Train ce loss 0.5174033164978027
Total val top-k-accuracy: 0.006020930232558141
Train ce loss 0.48974930346012113
Total val top-k-accuracy: 0.005987596899224808
Train ce loss 0.4794753432273865
Total val top-k-accuracy: 0.006120930232558141


  0%|          | 0/79 [00:00<?, ?it/s]

Train ce loss 0.04465245902538299
Total val top-k-accuracy: 0.006187596899224808
Train ce loss 0.43614035546779634
Total val top-k-accuracy: 0.006087596899224807
Train ce loss 0.42870389819145205
Total val top-k-accuracy: 0.006087596899224808
Train ce loss 0.41297824680805206
Total val top-k-accuracy: 0.005954263565891475
Train ce loss 0.4067755460739136
Total val top-k-accuracy: 0.0058542635658914745
Train ce loss 0.4057333916425705
Total val top-k-accuracy: 0.005920930232558142
Train ce loss 0.39246247708797455
Total val top-k-accuracy: 0.005798449612403102
Train ce loss 0.4003187656402588
Total val top-k-accuracy: 0.0059984496124031025


  0%|          | 0/79 [00:00<?, ?it/s]

...

Train ce loss 0.03706707656383514
Total val top-k-accuracy: 0.0058426356589147315
Train ce loss 0.35508624017238616
Total val top-k-accuracy: 0.005698449612403103
Train ce loss 0.36355395019054415
Total val top-k-accuracy: 0.00566511627906977
Train ce loss 0.35322476625442506
Total val top-k-accuracy: 0.005875968992248065
Train ce loss 0.3552398681640625
Total val top-k-accuracy: 0.005809302325581398
Train ce loss 0.3609453529119492
Total val top-k-accuracy: 0.0058759689922480655
Train ce loss 0.34745472073554995
Total val top-k-accuracy: 0.005775968992248065
Train ce loss 0.3584765285253525
Total val top-k-accuracy: 0.005709302325581398


  0%|          | 0/79 [00:00<?, ?it/s]

Train ce loss 0.03515855371952057
Total val top-k-accuracy: 0.005531782945736435
Train ce loss 0.34079441130161287
Total val top-k-accuracy: 0.0056317829457364355
Train ce loss 0.34976442754268644
Total val top-k-accuracy: 0.005731782945736436
Train ce loss 0.33919302225112913
Total val top-k-accuracy: 0.005865116279069769
Train ce loss 0.3431587964296341
Total val top-k-accuracy: 0.005898449612403102
Train ce loss 0.349368491768837
Total val top-k-accuracy: 0.005998449612403104
Train ce loss 0.33714440166950227
Total val top-k-accuracy: 0.006109302325581398
Train ce loss 0.3477638840675354
Total val top-k-accuracy: 0.005865116279069771


  0%|          | 0/79 [00:00<?, ?it/s]

Train ce loss 0.03428495824337006
Total val top-k-accuracy: 0.005931782945736438
Train ce loss 0.3324308395385742
Total val top-k-accuracy: 0.005765116279069771
Train ce loss 0.34135088324546814
Total val top-k-accuracy: 0.005865116279069771
Train ce loss 0.32992146015167234
Total val top-k-accuracy: 0.005865116279069771
Train ce loss 0.3336073666810989
Total val top-k-accuracy: 0.005798449612403104
Train ce loss 0.3395779490470886
Total val top-k-accuracy: 0.0058759689922480655
Train ce loss 0.3284361958503723
Total val top-k-accuracy: 0.005975968992248064
Train ce loss 0.33770278096199036
Total val top-k-accuracy: 0.005909302325581399
Elapsed time: 1242.2327511310577 seconds


In [18]:
torch.save(model.cpu().state_dict(),'two_tower_movie.pt')

# Redis Vector Database

For more information on Redis Vector Database, please visit

Redis as a vector database quick start guide

Getting Started with RedisVL

In [19]:
!nohup udocker --allow-root run -p 6379:6379 redis/redis-stack-server &

nohup: appending output to 'nohup.out'


In [21]:
%%sh

while true
do
  line=`ps -ef |grep redis | wc -l`
  if [ $line -eq 2 ]; then
    echo "Redis is running!"

    break
  fi
  sleep 5
  #echo "waiting for redis"
done

Redis is running!


In [ ]:
#model=cls_model(943,1682)
#model.load_state_dict(torch.load('two_tower_movie.pt'))

In [22]:
from redisvl.schema import IndexSchema

### Define Redis Index

This redisvl schema defines an index for storing and retrieving movie-related data efficiently in Redis using vector search capabilities.



In [23]:

schema = IndexSchema.from_dict({
    "index": {
        "name": "movie-idx",
        "prefix": "movie",
        "storage_type": "json"
    },
    "fields": [
        {"name": "movie", "type": "tag"},
        {
            "name": "embedding",
            "type": "vector",
            "attrs": {
                "algorithm": "flat",
                "datatype": "float32",
                "dims": 16,
                "distance_metric": "l2"
            }
        }
    ]
})

### Create Redis Index

In [24]:

from redis import Redis
from redisvl.index import SearchIndex

# Establish Redis connection and define index
client = Redis.from_url("redis://localhost:6379")
index = SearchIndex(schema, client)

# Create the index in Redis
#index.delete()
index.create()


### Load all the Item(movie) in the Redis DB

In [25]:

ind=0
for movie in model.ItemEmbedding.weight.data:
  data = {"movie": "movie:"+str(ind),"embedding": movie.tolist() }
  index.load([data], id_field="movie")
  ind+=1

### Recommandation for User 1

In [53]:
from redisvl.query import VectorQuery

user_id=1
users=[user_id-1]
movie_list=[]
for user in model.UserEmbedding.weight.data[users]:
    print(f"user: {user}")
    query = VectorQuery(
       vector=user.tolist(),
       vector_field_name="embedding",
       num_results=5
    )

    results = index.query(query)
    for result in results:
      movie_id = result['id'].split(':')[-1]
      print(movie_id)
      movie_list.append(int(movie_id))

user: tensor([ 0.2486, -1.8027, -1.4350, -0.9774, -0.9410, -1.4452,  2.1552, -0.7900,
        -0.6998, -0.1488, -0.7016,  0.8483,  1.4505,  0.3350, -0.7079, -0.5275])
1216
1486
1074
283
647


In [56]:
data=tuple(movie_list)
spark.sql(f"""select * from items where movie_id in {data}""").show()

+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|movie_id|         movie_title|release_date|video_release_date|            imdb_url|unknown|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film_Noir|Horror|Musical|Mystery|Romance|Sci_Fi|Thriller|War|Western|
+--------+--------------------+------------+------------------+--------------------+-------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+-------+-------+-------+------+--------+---+-------+
|     283|         Emma (1996)| 02-Aug-1996|              NULL|http://us.imdb.co...|      0|     0|        0|        0|       0|     0|    0|          0|    1|      0|        0|     0|      0|      0|      1|     0|       0|  0|      0|
|     647|          Ran (1985)| 01-Jan-1985|        